In [1]:
import time
import torch
import wandb

def log_model_performance(encoder, batch_tensor, runs: int = 100):
    device = batch_tensor.device
    encoder.eval()

    # Warm-up
    with torch.no_grad():
        _ = encoder(batch_tensor)

    torch.cuda.synchronize(device)
    total_time = 0
    batch_size = batch_tensor.size(0)   

    with torch.no_grad():
        for step in range(runs):
            start = time.time()
            _ = encoder(batch_tensor)
            torch.cuda.synchronize(device)
            end = time.time()
            total_time = (end - start)
            throughput = batch_size / total_time

            wandb.log({
                f"throughput_fps": throughput
            },step=step)

In [6]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

# Logging MobileNetV3-Large

In [7]:
from cnn_gemma import CNNImageEncoder, CNNGemmaConfig
import json

config_folder = "..\src\configs"

In [ ]:
with open(os.path.join(config_folder, "mobilenet_multiple.json"), "r") as f:
    model_config_file = json.load(f)
    config = CNNGemmaConfig(**model_config_file)
    encoder_config = config.vision_config

mobilenet_encoder = CNNImageEncoder(config=encoder_config)

In [ ]:
import torch

batch_size = 32
channels = 3
height = 224
width = 224

dummy_batch = torch.rand(batch_size, channels, height, width).to("cuda")
mobilenet_encoder.eval().to("cuda")
import wandb
wandb.init(
    project="Final-Project",
    name="mobilenetv3_run",
    config={"encoder": "mobilenetv3", "batch_size": batch_size}
)

In [ ]:
log_model_performance(encoder=mobilenet_encoder,batch_tensor=dummy_batch)

# Logging EfficientNet

In [ ]:
with open(os.path.join(config_folder, "efficientnet_multiple.json"), "r") as f:
    model_config_file = json.load(f)
    config = CNNGemmaConfig(**model_config_file)
    encoder_config = config.vision_config

efficientnet_encoder = CNNImageEncoder(config=encoder_config)

In [ ]:
import torch

batch_size = 32
channels = 3
height = 224
width = 224

dummy_batch = torch.rand(batch_size, channels, height, width).to("cuda")
efficientnet_encoder.eval().to("cuda")
import wandb
wandb.init(
    project="Final-Project",
    name="efficientnetb0_run",
    config={"encoder": "efficientnetb0", "batch_size": batch_size}
)

In [ ]:
log_model_performance(encoder=efficientnet_encoder,batch_tensor=dummy_batch)

# Logging Siglip

In [ ]:
from siglip import SiglipVisionModel, SiglipVisionConfig
import torch

batch_size = 32
channels = 3
height = 224
width = 224

dummy_batch = torch.rand(batch_size, channels, height, width).to("cuda")
siglip_encoder = SiglipVisionModel(config=SiglipVisionConfig())
siglip_encoder.eval().to("cuda")
import wandb
wandb.init(
    project="Final-Project",
    name="siglip_run",
    config={"encoder": "siglip", "batch_size": batch_size}
)

In [ ]:
log_model_performance(encoder=siglip_encoder,batch_tensor=dummy_batch)

In [21]:
wandb.finish()

throughput_fps,▃▄█▂▇▆▄▁▄▅▃▅▅▃▃▅▆▄▅▄▂▅▅▃▅▅▅▆▆▆▅▇▇▃▆▃▇▄▆▅
throughput_fps,219.40358
